In [7]:
#!/usr/bin/env python
# coding: utf-8

import sys, os, git
import shutil
import numpy as np
import subprocess

from utilities import detect_saving_folder, try_to_copy_file, \
                      return_file_content, detect_block_code, \
                      detect_block_types, create_file, detect_last_matching_line \

current_path = os.getcwd()
git_repo = git.Repo(current_path, search_parent_directories=True)
git_path = git_repo.git.rev_parse("--show-toplevel")

path_to_docs = git_path + "/docs/source/chapters/"

# make sure the documentaiton was found
assert os.path.exists(path_to_docs), """Documentation files not found"""

if os.path.exists("generated-codes/") is False:
    os.mkdir("generated-codes/")

In [42]:
# In chapter 1, all the files are simply created.
for chapter_id in [1, 2]: # np.arange(10):
    filename = path_to_docs + "chapter"+str(chapter_id)+".rst"
    created_files = []
    if os.path.exists(filename):
        folder = detect_saving_folder(chapter_id)
        created_files = try_to_copy_file(chapter_id, created_files)
        file_content = return_file_content(filename)
        block_contents, block_names = detect_block_code(file_content)
        block_types = detect_block_types(block_contents)
        created_files = create_file(block_contents, block_names, created_files, folder)


for content, name, type in zip(block_contents, block_names, block_types):
    if "test_" not in name:
        ISIMPORT, ISMETHOD, ISCLASS, ISPARTIAL, ISINIT = type
        original_file_content = return_file_content(folder+name+".py")
        if ISIMPORT:
            # Add the content at the start of the file
            file = open(folder+name+".py", "w")
            for line in content:
                file.write(line)
            for line in original_file_content:
                file.write(line)
            file.close()
        elif (ISMETHOD) & (ISPARTIAL is False):
            # Add the content at the end of the file, with an indentation
            file = open(folder+name+".py", "w")
            for line in original_file_content:
                file.write(line)
            for line in content:
                file.write("    "+line)
            file.close()
        elif ISPARTIAL:
            # Add the content to the end of the __init__ method
            cpt_new_last, cpt_old_last = detect_last_matching_line(content, original_file_content)
            file = open(folder+name+".py", "w")
            for line in original_file_content[:cpt_old_last]:
                file.write(line)
            for line in content[cpt_new_last-1:]:
                if "(...)" not in line:
                    file.write("    "+line)
            for line in original_file_content[cpt_old_last:]:
                file.write(line)
            file.close()

        else:
            print(ISIMPORT, ISMETHOD, ISCLASS, ISPARTIAL, ISINIT)
            for line in content:
                print(line[:-1])
            print()

False False True False True


class Prepare:
    def __init__(self,
                number_atoms=[10],  # List
                epsilon=[0.1],  # List - Kcal/mol
                sigma=[1],  # List - Angstrom
                atom_mass=[1],  # List - g/mol
                *args,
                **kwargs):
        self.number_atoms = number_atoms
        self.epsilon = epsilon
        self.sigma = sigma
        self.atom_mass = atom_mass
        super().__init__(*args, **kwargs)




In [38]:
content

['\n',
 '\n',
 'def __init__(self,\n',
 '    (...)\n',
 '    super().__init__(*args, **kwargs)\n',
 '    self.calculate_LJunits_prefactors()\n',
 '\n']

In [43]:
name = "Prepare"
# Remove doublons in init functions
original_class = open(folder+name+".py", "r")
original_start_init = None
original_end_init = []
original_content = []
for cpt, l in enumerate(original_class):
    original_content.append(l)
    if ("def" in l) & ("__init__" in l):
        original_start_init = cpt
    elif (":" in l) & ("def" in l) & ("__init__" not in l):
        original_end_init.append(cpt)        
original_end_init = original_end_init[0]
original_class.close()

# detect unique lines
_, idx = np.unique(original_content[original_start_init:original_end_init], return_index=True)
new_bloc = []
for i in np.sort(idx):
    new_bloc.append(original_content[original_start_init:original_end_init][i])

REPLACED = False
new_class = open(folder+name+".py", "w")
for cpt, l in enumerate(original_content):
    if (cpt < original_start_init) | (cpt > original_end_init-2):
        new_class.write(l)
    else:
        if REPLACED is False:
            REPLACED = True
            for ll in new_bloc:
                new_class.write(ll)
new_class.close()

In [33]:
new_bloc


['    def __init__(self,\n',
 '                *args,\n',
 '                **kwargs):\n',
 '        super().__init__(*args, **kwargs)\n',
 '        self.calculate_LJunits_prefactors()\n',
 '        self.nondimensionalize_units_0()\n',
 '        self.identify_atom_properties()\n',
 '    \n',
 '\n']

In [5]:
cpt_new_last, cpt_old_last

(4, 11)

In [ ]:
new_line

'\n'

In [ ]:
content

In [ ]:
end_init = original_empty_lines[original_empty_lines
                                > original_position_class][0]
new_class = open(folder+class_name_i+".py", "w")
for cpt, l in enumerate(original_content):
    new_class.write(l)
    if cpt == end_init+1:
        for ll in myclass:
            # Make sure the line is not already in
            # before writting it
            # ALREADYIN = False
            # for lll in original_content:
            #     if ll in lll:
            #         ALREADYIN = True
            if ("def __init__" not in ll) & ("(...)" not in ll): #  & (ALREADYIN is False):
                new_class.write("    "+ll)

new_class.close()

In [ ]:
ISIMPORT = False
ISMETHOD = False
ISCLASS = False
ISINIT = False
PARTIAL = False
new_position_class = None
new_position_partial = None
for cpt, l in enumerate(myclass):
    if (("import" in l) & ("as" in l)) | (("from" in l) & ("import" in l)):
        ISIMPORT = True
    if ("def" in l) & ("(self" in l) & ("):" in l):
        ISMETHOD = True
    if ("class" in l) & (":" in l):
        ISCLASS = True
        new_position_class = cpt
    if ("def" in l) & ("__init__" in l):
        ISINIT = True
    if ("(...)" in l):
        PARTIAL = True
        new_position_partial = cpt

In [ ]:

    file = open(filename, "r")
    print_file = False
    for line in file: # Loop over all the lines of the file
        if ".. label::" in line: # Detect the label "start" and label "end"
            label = line.split(".. label:: ")[1] # Look for label in the line
            if label[:6] == "start_": # Detect starting label
                class_name_i = label.split("start_")[1].split("_class")[0]
                if "test" not in class_name_i:
                    list_files.append(class_name_i+".py")
                    print_file = True
                    # create file
                    myclass = open(folder+class_name_i+".py", "w")
            elif label[:4] == "end_": # Detect ending label
                class_name_f = label.split("end_")[1].split("_class")[0]
                if "test" not in class_name_f:
                    assert class_name_f == class_name_i, """Different class closed, inconsistency in rst file?"""
                    print_file = False
                    # close file
                    myclass.close()
        else:
            if print_file: # Print the content of the label into files
                if ".. code-block::" not in line: # Ignore code block line
                    if len(line) > 1: # Remove the indentation
                        myclass.write(line[4:])
                    else:
                        myclass.write(line)


In [ ]:
for chapter_id in np.arange(2, 10):
    filename = path_to_docs + "chapter"+str(chapter_id)+".rst"
    list_classes = []
    if os.path.exists(filename):
        # saving folder
        folder = "generated-codes/chapter"+str(chapter_id)+"/"
        previous_folder = "generated-codes/chapter"+str(chapter_id-1)+"/"
        if os.path.exists(folder) is False:
            os.mkdir(folder)
        # copy all the files from the previous chapter
        for file in list_files:
            shutil.copyfile(previous_folder+"/"+file, folder+"/"+file)
        print_file = False
        for line in open(filename, "r"): # Loop over all the lines of the file
            if ".. label::" in line: # Detect the label "start" and label "end"
                label = line.split(".. label:: ")[1] # Look for label in the line
                if label[:6] == "start_": # Detect starting label
                    class_name_i = label.split("start_")[1].split("_class")[0]
                    if "test" not in class_name_i:
                        list_classes.append(class_name_i)
                        print_file = True
                        myclass = []
                elif label[:4] == "end_": # Detect ending label
                    class_name_f = label.split("end_")[1].split("_class")[0]
                    if "test" not in class_name_f:
                        assert class_name_f == class_name_i, """Different class closed, inconsistency in rst file?"""
                        print_file = False
                        # myclass.close()
                        if (len(myclass) > 0) & ("test" not in class_name_f):
                            # detect the type of code
                            ISIMPORT = False
                            ISMETHOD = False
                            ISCLASS = False
                            ISINIT = False
                            PARTIAL = False
                            new_position_class = None
                            new_position_partial = None
                            for cpt, l in enumerate(myclass):
                                if (("import" in l) & ("as" in l)) | (("from" in l) & ("import" in l)):
                                    ISIMPORT = True
                                if ("def" in l) & ("(self" in l) & ("):" in l):
                                    ISMETHOD = True
                                if ("class" in l) & (":" in l):
                                    ISCLASS = True
                                    new_position_class = cpt
                                if ("def" in l) & ("__init__" in l):
                                    ISINIT = True
                                if ("(...)" in l):
                                    PARTIAL = True
                                    new_position_partial = cpt

                            original_class = open(folder+class_name_i+".py", "r")
                            original_content = []
                            original_position_class = None
                            original_empty_lines = []
                            for cpt, l in enumerate(original_class):
                                original_content.append(l)
                                if "__init__" in l:
                                    original_position_init = cpt
                                elif "class" in l:
                                    original_position_class = cpt
                                elif "\n" == l:
                                    original_empty_lines.append(cpt)
                            original_empty_lines = np.array(original_empty_lines)
                            if ISIMPORT:
                                new_class = open(folder+class_name_i+".py", "w")
                                for l in myclass:
                                    new_class.write(l)
                                for l in original_content:
                                    new_class.write(l)
                                new_class.close()
                            elif ISMETHOD:
                                new_class = open(folder+class_name_i+".py", "w")
                                for l in original_content:
                                    new_class.write(l)
                                for l in myclass:
                                    new_class.write("    "+l)
                                new_class.close()
                            elif (ISCLASS) & (PARTIAL is False):
                                new_class = open(folder+class_name_i+".py", "w")
                                REPLACE = False
                                REPLACED = False
                                for cpt, l in enumerate(original_content):
                                    if cpt == original_position_class: # start class
                                        REPLACE = True
                                    elif (l == "\n") & REPLACE:
                                        REPLACE = False

                                    if REPLACE:
                                        if REPLACED is False:
                                            REPLACED = True
                                            for ll in myclass:
                                                new_class.write(ll)
                                    else:
                                        new_class.write(l)
                                new_class.close()
                            elif (ISINIT) & (PARTIAL):
                                end_init = original_empty_lines[original_empty_lines
                                                                > original_position_class][0]
                                new_class = open(folder+class_name_i+".py", "w")
                                for cpt, l in enumerate(original_content):
                                    new_class.write(l)
                                    if cpt == end_init+1:
                                        for ll in myclass:
                                            # Make sure the line is not already in
                                            # before writting it
                                            # ALREADYIN = False
                                            # for lll in original_content:
                                            #     if ll in lll:
                                            #         ALREADYIN = True
                                            if ("def __init__" not in ll) & ("(...)" not in ll): #  & (ALREADYIN is False):
                                                new_class.write("    "+ll)

                                new_class.close()

                            # remove space in empty lines
                            original_class = open(folder+class_name_i+".py", "r")
                            original_content = []
                            for cpt, l in enumerate(original_class):
                                original_content.append(l)
                            original_class.close()
                            new_class = open(folder+class_name_i+".py", "w")
                            for l in original_content:
                                if l == "    \n":
                                    l = "\n"
                                new_class.write(l)
                            new_class.close()
            else:
                if print_file: # Print the content of the label into files
                    if ".. code-block::" not in line: # Ignore code block line
                        if len(line) > 1: # Remove the indentation
                            # myclass.write(line[4:])
                            myclass.append(line[4:])
                        else:
                            # myclass.write(line)
                            myclass.append(line)

        for class_name in np.unique(list_classes):

            # Remove doublons in init functions
            original_class = open(folder+class_name+".py", "r")
            original_start_init = None
            original_end_init = []
            original_content = []
            for cpt, l in enumerate(original_class):
                original_content.append(l)
                if ("def" in l) & ("__init__" in l):
                    original_start_init = cpt
                elif (":" in l) & ("def" in l) & ("__init__" not in l):
                    original_end_init.append(cpt)        
            original_end_init = original_end_init[0]
            original_class.close()

            # detect unique lines
            _, idx = np.unique(original_content[original_start_init:original_end_init], return_index=True)
            new_bloc = []
            for i in np.sort(idx):
                new_bloc.append(original_content[original_start_init:original_end_init][i])

            REPLACED = False
            new_class = open(folder+class_name+".py", "w")
            for cpt, l in enumerate(original_content):
                if (cpt < original_start_init) | (cpt > original_end_init-2):
                    new_class.write(l)
                else:
                    if REPLACED is False:
                        REPLACED = True
                        for ll in new_bloc:
                            new_class.write(ll)

            new_class.close()

In [ ]:
# Detect test files
for chapter_id in np.arange(1, 10):
    filename = path_to_docs + "chapter"+str(chapter_id)+".rst"
    test_files = []
    if os.path.exists(filename):
        # saving folder
        folder = "generated-codes/chapter"+str(chapter_id)+"/"
        if os.path.exists(folder) is False:
            os.mkdir(folder)
        file = open(filename, "r")
        print_file = False
        for line in file: # Loop over all the lines of the file
            if ".. label::" in line: # Detect the label "start" and label "end"
                label = line.split(".. label:: ")[1] # Look for label in the line
                if label[:6] == "start_": # Detect starting label
                    class_name_i = label.split("start_")[1].split("_class")[0]
                    if "test" in class_name_i:
                        test_files.append(class_name_i+".py")
                        print_file = True
                        # create file
                        myclass = open(folder+class_name_i+".py", "w")
                elif label[:4] == "end_": # Detect ending label
                    class_name_f = label.split("end_")[1].split("_class")[0]
                    if "test" in class_name_f:
                        assert class_name_f == class_name_i, """Different class closed, inconsistency in rst file?"""
                        print_file = False
                        # close file
                        myclass.close()
            else:
                if print_file: # Print the content of the label into files
                    if ".. code-block::" not in line: # Ignore code block line
                        if len(line) > 1: # Remove the indentation
                            myclass.write(line[4:])
                        else:
                            myclass.write(line)


In [ ]:
# Run test files
for chapter_id in np.arange(1, 10):
    # Test
    mycwd = os.getcwd()
    os.chdir(folder)
    for test_file in test_files:
        print("TEST", "chapter"+str(chapter_id)+".rst", test_file)
        subprocess.call(["python3", test_file])
    os.chdir(mycwd)